In [1]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 28.0 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=291de8cd1ab7850651e0844bdacfaa2b8f47fd285b1bb60ded77b7e33aacaa20
  Stored in directory: /Users/sohamshah/Library/Caches/pip/wheels/cf/c0/b9/f147f4220fd1d9277d0981b88b35b26f03ad910fffd60013a6
Successfully built pyspark
Note: you may need to restart the kernel to use updated packages.


In [29]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import sum, count

In [2]:
spark = SparkSession.builder.appName("CSV Loader").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/06 09:23:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_RS = spark.read.csv('Retail_sales_data.csv', header=True, inferSchema=True)

24/09/06 09:23:57 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [5]:
df_RS.show(5)

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
only showing top 5 rows



In [8]:
df_RS.describe().show()

24/09/06 09:44:01 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+------------------+------------------+--------------------+------------------+---------------+-----------------+------------------+-----------+
|summary|         InvoiceNo|         StockCode|         Description|          Quantity|    InvoiceDate|        UnitPrice|        CustomerID|    Country|
+-------+------------------+------------------+--------------------+------------------+---------------+-----------------+------------------+-----------+
|  count|            541909|            541909|              540455|            541909|         541909|           541909|            406829|     541909|
|   mean|  559965.752026781|27623.240210938104|             20713.0|  9.55224954743324|           NULL|4.611113626088366|15287.690570239585|       NULL|
| stddev|13428.417280798714| 16799.73762842762|                NULL|218.08115785023472|           NULL|96.75985306118037|1713.6003033215961|       NULL|
|    min|            536365|             10002| 4 PURPLE FLOCK D...|            -8

In [11]:
df_RS.printSchema()


root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)



In [12]:
df_RS_cleaned = df_RS.fillna({'Description': 'Unknown Item'})

In [ ]:
###Dropping Customer_ID column as they are a lot of NUll values and it is not required for Analysis######

In [13]:
df_cleaned_final = df_RS_cleaned.drop('CustomerID')

In [15]:
df_cleaned_final.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = false)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- Country: string (nullable = true)



In [81]:
from pyspark.sql.functions import trim
df_trimmed_RS = df_cleaned_final.withColumn("Description_trimmed", trim(df_cleaned_final["Description"]))

In [82]:
df_trimmed_RS.show()

+---------+---------+--------------------+--------+--------------+---------+--------------+--------------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|       Country| Description_trimmed|
+---------+---------+--------------------+--------+--------------+---------+--------------+--------------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|United Kingdom|WHITE HANGING HEA...|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|United Kingdom| WHITE METAL LANTERN|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|United Kingdom|CREAM CUPID HEART...|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|United Kingdom|KNITTED UNION FLA...|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|United Kingdom|RED WOOLLY HOTTIE...|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|12/1/2010 8:26|     7.65|United Kingdom|SET 7

In [83]:
from pyspark.sql.functions import split
split_col = split(df_trimmed_RS["InvoiceDate"], " ")

In [84]:
df_Date = df_trimmed_RS.withColumn("InvDate", split_col.getItem(0))
df_Date.show()

+---------+---------+--------------------+--------+--------------+---------+--------------+--------------------+---------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|       Country| Description_trimmed|  InvDate|
+---------+---------+--------------------+--------+--------------+---------+--------------+--------------------+---------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|United Kingdom|WHITE HANGING HEA...|12/1/2010|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|United Kingdom| WHITE METAL LANTERN|12/1/2010|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|United Kingdom|CREAM CUPID HEART...|12/1/2010|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|United Kingdom|KNITTED UNION FLA...|12/1/2010|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|United Kingdom|RED WOOLLY HOTTIE...|12/1/2010|
|   536365|    2

In [85]:
df_RS_final = df_Date.withColumn("UnitPrice", df_Date["UnitPrice"].cast("integer"))

In [86]:
df_RS_final.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = false)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- Description_trimmed: string (nullable = false)
 |-- InvDate: string (nullable = true)



In [91]:
df_Retail = df_RS_final.withColumn("Sale_amount", col("UnitPrice") * col("Quantity"))

In [92]:
df_Retail.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = false)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- Description_trimmed: string (nullable = false)
 |-- InvDate: string (nullable = true)
 |-- Sale_amount: integer (nullable = true)



In [93]:
sales_summary = df_Retail.groupBy('InvDate') \
    .agg(
        sum("Sale_amount").alias("total_sales"),
        count('InvoiceNo').alias("number_of_transactions")
    )

In [94]:
sales_summary.show()

+---------+-----------+----------------------+
|  InvDate|total_sales|number_of_transactions|
+---------+-----------+----------------------+
|3/23/2011|      16558|                  1319|
|2/16/2011|      17046|                  1191|
|2/10/2011|       8827|                   785|
|3/13/2011|       2664|                   537|
|1/20/2011|      12447|                  1502|
| 2/9/2011|      13033|                   879|
|3/25/2011|      23030|                  1386|
|1/30/2011|       4465|                   722|
| 2/8/2011|      15597|                  1228|
|1/17/2011|      21989|                  2557|
|1/10/2011|      17498|                  1976|
|1/13/2011|      14732|                  1445|
|3/21/2011|      10649|                  1068|
|1/16/2011|       4887|                   646|
|1/18/2011|      14378|                  1447|
|2/21/2011|      12567|                  1425|
| 1/5/2011|     -11751|                  1743|
|2/27/2011|       6564|                   812|
|1/12/2011|  

In [95]:
df_Retail.show()

+---------+---------+--------------------+--------+--------------+---------+--------------+--------------------+---------+-----------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|       Country| Description_trimmed|  InvDate|Sale_amount|
+---------+---------+--------------------+--------+--------------+---------+--------------+--------------------+---------+-----------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|        2|United Kingdom|WHITE HANGING HEA...|12/1/2010|         12|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|        3|United Kingdom| WHITE METAL LANTERN|12/1/2010|         18|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|        2|United Kingdom|CREAM CUPID HEART...|12/1/2010|         16|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|        3|United Kingdom|KNITTED UNION FLA...|12/1/2010|         18|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1

In [ ]:
#####Top 10 products with the highest Sales######

In [96]:
from pyspark.sql.functions import sum
Prod_sales = df_Retail.groupBy("Description") \
    .agg(sum("Sale_amount").alias("total_sales"))

In [97]:
top_products = Prod_sales.orderBy("total_sales", ascending=False).limit(10)
top_products.show()

+--------------------+-----------+
|         Description|total_sales|
+--------------------+-----------+
|      DOTCOM POSTAGE|     205896|
|REGENCY CAKESTAND...|     153384|
|       PARTY BUNTING|      87526|
|WHITE HANGING HEA...|      77333|
|             POSTAGE|      66254|
|JUMBO BAG RED RET...|      65022|
|PAPER CHAIN KIT 5...|      50053|
|  RABBIT NIGHT LIGHT|      48719|
|       CHILLI LIGHTS|      46149|
|PICNIC BASKET WIC...|      39589|
+--------------------+-----------+



In [ ]:
######Saving the Transformed Data Back to a SQL Database######

In [101]:
db_url = "jdbc:sqlite:/Users/sohamshah/Downloads/Online_Retail.db"
db_properties = {
 "driver": "org.sqlite.JDBC"
}
df_Retail.write.jdbc(
    url=db_url,
    table="Retail_Sales",
    mode="overwrite",        
    properties=db_properties
)


Py4JJavaError: An error occurred while calling o312.jdbc.
: java.lang.ClassNotFoundException: org.sqlite.JDBC
	at java.base/java.net.URLClassLoader.findClass(URLClassLoader.java:445)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:592)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:525)
	at org.apache.spark.sql.execution.datasources.jdbc.DriverRegistry$.register(DriverRegistry.scala:46)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1(JDBCOptions.scala:103)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1$adapted(JDBCOptions.scala:103)
	at scala.Option.foreach(Option.scala:407)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:103)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:254)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:258)
	at org.apac